<a href="https://colab.research.google.com/github/cardstud/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
# Save file to notebook
from google.colab import files
uploaded = files.upload()

Saving house-votes-84.csv to house-votes-84 (4).csv


In [0]:
# Import data analysis and visualization libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

### Attribute Information:

0. Class Name: 2 (democrat, republican)
1. handicapped-infants: 2 (y,n)
2. water-project-cost-sharing: 2 (y,n)
3. adoption-of-the-budget-resolution: 2 (y,n)
4. physician-fee-freeze: 2 (y,n)
5. el-salvador-aid: 2 (y,n)
6. religious-groups-in-schools: 2 (y,n)
7. anti-satellite-test-ban: 2 (y,n)
8. aid-to-nicaraguan-contras: 2 (y,n)
9. mx-missile: 2 (y,n)
10. immigration: 2 (y,n)
11. synfuels-corporation-cutback: 2 (y,n)
12. education-spending: 2 (y,n)
13. superfund-right-to-sue: 2 (y,n)
14. crime: 2 (y,n)
15. duty-free-exports: 2 (y,n)
16. export-administration-act-south-africa: 2 (y,n)

In [0]:
# Save csv as a Pandas DataFrame: also, noticed header issue so fixed
df= pd.read_csv('house-votes-84.csv', header=None)

In [0]:
# Check head for issues
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
# Check tail
df.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
430,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
432,republican,n,?,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,?,?,?,?,n,y,y,y,n,y
434,republican,n,y,n,y,y,y,n,n,n,y,n,y,y,y,?,n


In [0]:
# Check random sample from df
df.sample(8)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
355,republican,y,n,y,y,n,n,n,y,y,y,n,n,n,y,y,y
406,democrat,y,n,y,n,y,y,n,n,y,y,n,n,y,y,n,y
239,republican,n,n,n,y,y,y,y,n,n,y,n,n,n,y,y,y
14,republican,n,y,n,y,y,y,n,n,n,n,n,y,?,?,n,?
75,democrat,n,y,y,y,y,y,n,n,n,y,y,n,y,y,n,n
232,democrat,n,y,y,n,n,n,y,y,n,n,y,n,n,n,y,?
294,democrat,n,y,y,n,n,y,y,y,n,y,y,n,y,y,n,?
316,democrat,n,n,n,n,n,y,n,y,y,n,y,y,y,y,y,n


In [0]:
# Check Shape and see if matches websites numbers (site has 435 observations, 17 total attributes -16 + target)
df.shape

(435, 17)

In [0]:
# Check for odd characters. Found ?. Replace with NaN values
df =df.replace('?', np.NaN)

In [0]:
# Check head again to see if values replaced. They were.
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [0]:
# Find sum of missing values
df.isna().sum()

0       0
1      12
2      48
3      11
4      11
5      15
6      11
7      14
8      15
9      22
10      7
11     21
12     31
13     25
14     17
15     28
16    104
dtype: int64

In [0]:
# Replace missing values - will put 'nv' (no vote) for all NaN values
df = df.fillna(0)

In [0]:
# Check to see if NaN changed to 'nv'
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,0,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,0
2,democrat,0,y,y,0,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,0,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,0,y,y,y,y


In [0]:
# Check some summary statistics 
df.describe(exclude='number')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
count,435,435,435,435,435,435,435,435,435,435,435,435,435,435,435,435,435
unique,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
top,democrat,n,y,y,n,y,y,y,y,y,y,n,n,y,y,n,y
freq,267,236,195,253,247,212,272,239,242,207,216,264,233,209,248,233,269


In [0]:
# Change n/y to binary
#df =df.replace('y', 1)
#df = df.replace('n', 0)

In [0]:
#df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,0,1,0,1,1,1,0,0,0,1,nv,1,1,1,0,1
1,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,nv
2,democrat,nv,1,1,nv,1,1,0,0,0,0,1,0,1,1,0,0
3,democrat,0,1,1,0,nv,1,0,0,0,0,1,0,1,0,0,1
4,democrat,1,1,1,0,1,1,0,0,0,0,1,nv,1,1,1,1


In [0]:
# Change column names
df.columns = ['class', 'infants', 'water_cost', 'budget', 'fee_freeze', 'aid_elsalvador', 'rel_school', 'satellite', 'aid_contras', 'mx_missle', 'immigration', 'cutback', 'education', 'right_to_sue', 'crime', 'duty_free_ex', 'export_south_africa']

In [0]:
df.head()

,class,infants,water_cost,budget,fee_freeze,aid_elsalvador,rel_school,satellite,aid_contras,mx_missle,immigration,cutback,education,right_to_sue,crime,duty_free_ex,export_south_africa
0,republican,n,y,n,y,y,y,n,n,n,y,0,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,0
2,democrat,0,y,y,0,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,0,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,0,y,y,y,y


In [0]:
df.describe()

,class,infants,water_cost,budget,fee_freeze,aid_elsalvador,rel_school,satellite,aid_contras,mx_missle,immigration,cutback,education,right_to_sue,crime,duty_free_ex,export_south_africa
count,435,435,435,435,435,435,435,435,435,435,435,435,435,435,435,435,435
unique,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
top,democrat,n,y,y,n,y,y,y,y,y,y,n,n,y,y,n,y
freq,267,236,195,253,247,212,272,239,242,207,216,264,233,209,248,233,269


In [0]:
df.head()

,class,infants,water_cost,budget,fee_freeze,aid_elsalvador,rel_school,satellite,aid_contras,mx_missle,immigration,cutback,education,right_to_sue,crime,duty_free_ex,export_south_africa
0,republican,n,y,n,y,y,y,n,n,n,y,0,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,0
2,democrat,0,y,y,0,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,0,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,0,y,y,y,y


In [0]:
df_republican = df[df['class']== 'republican']

In [0]:
df_republican.shape


(168, 17)

In [0]:
df_democrat = df[df['class']== 'democrat']

In [0]:
df_democrat.shape

(267, 17)

In [0]:
df.shape

(435, 17)

In [0]:
df_republican_infants_y = df_republican[df_republican['infants'] == 'y']

In [0]:
df_republican_infants_y.sum()

class             republicanrepublicanrepublicanrepublicanrepubl...
infants                             yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
budget                              nnnnnyyynnnnyynnnnnynnnynnnnyny
fee_freeze                          yyyyyyyyyyyyyyyyynyyyyyyyyyyyyy
aid_elsalvador                      yyyyynyyyyyyyyyyynyyyyynyyyyyyy
rel_school                          nyyyynnnyyyyyyyyynnyyyynyyyyyyy
satellite                           ynnnyyyynnnyyyyynyyynnnnnnnnyny
mx_missle                           ynnnnyynnnnnnnnnnynnnnnynnnnnnn
immigration                         nynynyynnynnyynyyyynynyyynnyyyy
crime                               yyyyyyyyyyyyyyyyyyyyyyyynyyyyyy
dtype: object